In [1]:
import os
from dotenv import load_dotenv ,find_dotenv
_ = load_dotenv(find_dotenv())
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq

In [2]:
NEO4J_URI=os.getenv('NEO4J_URI')
NEO4J_USERNAME=os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD=os.getenv('NEO4J_PASSWORD')

In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [36]:
## menmory
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [4]:
loader=WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post_title","post-header")
            
            
        )
    ),
)
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
#documnet into chunks or splits
splits=text_splitter.split_documents(docs)

In [5]:
model="BAAI/bge-large-en"
embeddings = HuggingFaceEmbeddings(model_name=model)


c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#create a Neo4j vector store
neo4j_db=Neo4jVector.from_documents(
    splits,
    embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database='neo4j',
    index_name='lcel_index',
    node_label='lcel_node',
    text_node_property="text",
    embedding_node_property="embeddings",
      
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 1, column: 21, offset: 20} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`lcel_node` {id: row.id}) WITH c, row CALL db.create.setNodeVectorProperty(c, 'embeddings', row.embedding) SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS "


In [54]:
##load llm
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=GROQ_API_KEY,
    
)

In [ ]:
# window_memory = ConversationBufferWindowMemory(k=3)
# conversation_window = ConversationChain(
#     llm=llm, 
#     memory = window_memory,
#     verbose=True
# )

In [84]:
retriever = neo4j_db.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [85]:
response = rag_chain.invoke("What is Task Decomposition?")

print("\n----------\n")

print("Chains in RAG:")

print("\n----------\n")
print(response)

print("\n----------\n")


----------

Chains in RAG:

----------

Task decomposition is the process of breaking down a complex task into smaller, more manageable steps or subtasks. This can be done using various methods, including simple prompting, task-specific instructions, or human inputs. The goal of task decomposition is to enable an agent or model to plan ahead and tackle each step individually, making the overall task more achievable.

----------



c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


## memory

In [ ]:
# window_memory = ConversationBufferWindowMemory(k=3)
# conversation_window = ConversationChain(
#     llm=llm, 
#     memory = window_memory,
#     verbose=True
# )

C:\Users\kumar\AppData\Local\Temp\ipykernel_22300\784667498.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation_window = ConversationChain(


## old way[click](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/)

In [ ]:
#from langchain.chains import create_history_aware_retriever

In [ ]:
# from langchain.chains import (
#     StuffDocumentsChain, LLMChain, ConversationalRetrievalChain ,
# )
# chain=ConversationalRetrievalChain.from_llm(llm=llm,
#                                             retriever=retriever,
#                                             chain_type="stuff",
#                                             memory=window_memory,
#                                             return_source_documents=True,
#                                             get_chat_history=lambda h:h
#                                             )

In [86]:
from langchain_core.prompts import ChatPromptTemplate

# Define a function to extract the content of a message
def get_msg_content(msg):
    return msg.content

# Define the SYSTEM prompt for contextualizing the chat history to come up with a standalone question
contextualize_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

# Define the prompt for contextualizing the chat history to come up with a standalone question
contextualize_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_system_prompt),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

# Define the chain for contextualizing the chat history to come up with a standalone question
contextualize_chain = (
    contextualize_prompt
    | llm
    | get_msg_content
)

In [87]:
# Define the question-answering SYSTEM prompt to generate the final answer
qa_system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# Define the question-answering prompt to generate the final answer
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

# Define the chain to generate the final answer
qa_chain = (
    qa_prompt
    | llm
    | get_msg_content
) 
     


In [93]:
# from langchain.memory import ChatMessageHistory
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_core.runnables import chain
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [94]:
# Create a chat history object that can store messages in memory
chat_history = InMemoryChatMessageHistory()

In [95]:
# Define the overall chain the uses both the retrieved documents and the chat history to answer the question
@chain
def history_aware_qa(input):
     # Rephrase the question if needed
     if input.get('chat_history'):
         question = contextualize_chain.invoke(input)
     else:
         question = input['input']
    
     # Get context from the retriever
     context = retriever.invoke(question)

     # Get the final answer
     return qa_chain.invoke({
         **input,
         "context": context
     })

In [96]:
# Next, let’s incorporate stateful management of chat history and send the final prompt, 
# including chat history and retrieved context to the model for an output.
chat_history_for_chain = InMemoryChatMessageHistory()
qa_with_history = RunnableWithMessageHistory(
    history_aware_qa,
    lambda _: chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)
   

In [97]:
# Finally, let's invoke the chain
qa_with_history.invoke(
    {"input": "What is Task Decomposition?"},
    config={"configurable": {"session_id": "123"}},
)

c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


'Task decomposition is the process of breaking down a complicated task into smaller, manageable steps. It can be done using various methods, including simple prompting with a large language model (LLM), task-specific instructions, or human inputs.'

In [98]:
# Finally, let's invoke the chain
qa_with_history.invoke(
    {"input": "What is the previous question i asked?"},
    config={"configurable": {"session_id": "123"}},
)

c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


'Your previous question was "What is Task Decomposition?"'

In [99]:
# Finally, let's invoke the chain
qa_with_history.invoke(
    {"input": "My name is Lovakumar ,my age was 28"},
    config={"configurable": {"session_id": "123"}},
)

c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


"I don't know what to do with this information. It seems like you're introducing yourself, but I'm not sure how to respond or what to do with this information in the context of our conversation."

In [100]:
# Finally, let's invoke the chain
qa_with_history.invoke(
    {"input": "what is my name"},
    config={"configurable": {"session_id": "123"}},
)

c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


'Your name is Lovakumar.'